heartbeat thump?


In [ ]:
var fs = require('fs');
var path = require('path');

var CORE = path.resolve(path.join(__dirname, '../Core'));
var NODE = fs.existsSync('/usr/local/bin/node') ? '/usr/local/bin/node' : '/usr/bin/node';

function thump(thump, exit) {
    var out = fs.openSync('/tmp/myjob.log', 'a');
    var err = fs.openSync('/tmp/myjob.log', 'a');

    require('child_process').spawn(NODE, [
        '-e',
        'require(' + JSON.stringify(CORE) + ').import("' + thump + '")().catch(e => console.log(e)).then(() => process.exit())'
    ], {
        detached: true,
        stdio: [ 'ignore', out, err ]
    }).unref();
    if(typeof $$ !== 'undefined') {
        $$.done();
    }
    if(exit !== false) {
        process.exit(0);
    }
}
module.exports = thump;


run todays heartbeat items?


In [1]:
var importer = require('../Core');
var {
    getOauthClient,
    getDaysEvents,
    thump
} = importer.import([
    'days events',
    'import google calendar api',
    'heartbeat thump'
]);

var options = {
    calendarId: 'aws'
};

// test Google calendar API?
function runTodaysHeartbeat() {
    return (typeof options.auth === 'undefined'
           ? getOauthClient(options)
           : Promise.resolve([]))
        .then(() => getDaysEvents(new Date(), options))
        .then(r => {
            const heartbeat = r.filter(r => r.event.summary === 'heartbeat');
            const commands = heartbeat[0].event.description.split('\n').filter(e => e.trim().length > 0);
            commands.forEach(c => {
                thump(c, false);
            });
            if(typeof $$ !== 'undefined') {
                $$.done();
            }
            process.exit(0);
        })
}
module.exports = runTodaysHeartbeat;

if(typeof $$ !== 'undefined') {
    $$.async();
    runTodaysHeartbeat()
        .then(r => $$.sendResult(r))
        .then(e => $$.sendError(e))
}


reading notebook /Users/briancullinan/jupytangular2/Core/interpret all notebooks.ipynb
compiling /Users/briancullinan/jupytangular2/Core/interpret all notebooks.ipynb[0]
reading notebook /Users/briancullinan/jupytangular2/Core/interpret questions.ipynb
compiling /Users/briancullinan/jupytangular2/Core/interpret questions.ipynb[0]
compiling /Users/briancullinan/jupytangular2/Core/interpret questions.ipynb[1]
compiling /Users/briancullinan/jupytangular2/Core/interpret questions.ipynb[2]
compiling /Users/briancullinan/jupytangular2/Core/interpret questions.ipynb[3]
compiling /Users/briancullinan/jupytangular2/Core/interpret questions.ipynb[4]
compiling /Users/briancullinan/jupytangular2/Core/interpret all notebooks.ipynb[1]
reading notebook /Users/briancullinan/jupytangular2/Utilities/file system.ipynb
compiling /Users/briancullinan/jupytangular2/Utilities/file system.ipynb[7]
reading notebook /Users/briancullinan/jupytangular2/Analytics/budgeting.ipynb
reading notebook /Users/briancullin